In [6]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(
    llm= llm,
)

def add_message(input,output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Justddan, I live in South Korea","Wow tha is so cool!")

In [7]:
add_message("South Korea is so pretty","I wish I could go!!!")

In [8]:
get_history()

{'history': 'The human introduces themselves as Justddan and mentions that they live in South Korea. The AI responds by expressing admiration for this fact and expressing a desire to visit South Korea because it is so pretty.'}